In [3]:
from ilqr import iLQR
import gym
import numpy as np
import pandas as pd
import time

from aprl.agents import MujocoFiniteDiffDynamicsPerformance
from aprl.envs.mujoco_costs import get_cost
from experiments.common import set_seeds, make_env, fit_ilqr, \
                               on_iteration, receding, multi_evaluate

In [4]:
seed = 42
set_seeds(seed)

env_name = 'Swimmer-v2'
env, us_init = make_env(env_name, seed=seed)
dynamics = {'performance': MujocoFiniteDiffDynamicsPerformance(env)}
x0s = {k: dyn.get_state() for k, dyn in dynamics.items()}

costs = {k: get_cost(env_name) for k in dynamics.keys()}
ilqrs = {k: iLQR(dyn, costs[k], len(us_init))  
         for k, dyn in dynamics.items()}

In [5]:
xs, us = fit_ilqr(ilqrs, x0s, us_init)

dict_keys(['performance'])
*** Fitting performance ***
iteration -1 converged 17.60768730001879 [-0.20175069 -0.41185973  0.17787404 -0.97447278  1.32796911  0.20628244
  0.1305261  -0.12802691  0.56458499 -1.05335861] [-0.88589054 -0.43562585]
iteration 0 accepted 6.559545134130589 [-0.07628088 -0.41046617  0.34070365 -0.77293943  0.77608299  0.33660798
  0.11408165  0.07352318  0.85773864 -2.34829335] [-0.8835027  -0.39356047]
iteration 1 accepted 3.1283130073054237 [-0.04102583 -0.33531779  0.28753934 -0.61193955  0.51104277  0.2562309
  0.58013474 -0.36005006  1.78311332 -3.20959939] [-0.888962  -0.3922765]
iteration 2 accepted -66.94136561800778 [ 0.75398134  0.72315702 -1.436203    1.57079526 -0.04977283 -0.69570482
  0.26586472  0.7481155   0.68920547 -3.78811015] [-0.8626825 -0.392972 ]
iteration 3 failed -66.94136561800778 [ 0.75398134  0.72315702 -1.436203    1.57079526 -0.04977283 -0.69570482
  0.26586472  0.7481155   0.68920547 -3.78811015] [-0.8626825 -0.392972 ]
iteration

iteration 43 accepted -126.55420579291446 [ 1.59197202  0.32940179 -2.37862095  1.75132851  1.27796358  0.05736039
  0.3988127   0.59949173 -0.05762791 -1.45313717] [ 0.5203975  -0.61491543]
iteration 44 failed -126.55420579291436 [ 1.59197202  0.32940179 -2.37862095  1.75132851  1.27796358  0.05736039
  0.3988127   0.59949173 -0.05762791 -1.45313717] [ 0.5203975  -0.61491543]
iteration 45 failed -126.55420579291436 [ 1.59197202  0.32940179 -2.37862095  1.75132851  1.27796358  0.05736039
  0.3988127   0.59949173 -0.05762791 -1.45313717] [ 0.5203975  -0.61491543]
iteration 46 accepted -131.64695410966124 [ 1.64664326  0.24274451 -2.48223281  1.75359781  1.44624342  0.15644197
  0.26029626  0.38398137  0.04160193 -1.24197507] [ 0.42619154 -0.66543007]
iteration 47 accepted -132.02179681221986 [ 1.64726546  0.25759435 -2.47892546  1.73085817  1.4411175   0.59674352
 -0.10764961 -0.38806768  1.06452768 -1.87231953] [-0.45164797 -0.65185714]
iteration 48 failed -132.02179681221736 [ 1.64726

iteration 87 failed -143.30413096536384 [ 1.78936294  0.25490308 -2.49339679  1.76023696  1.43753312  0.19450627
  0.15263481  0.33484089  0.06286592 -1.59616612] [ 1.9138457 -0.1639374]
iteration 88 failed -143.30413096536384 [ 1.78936294  0.25490308 -2.49339679  1.76023696  1.43753312  0.19450627
  0.15263481  0.33484089  0.06286592 -1.59616612] [ 1.9138457 -0.1639374]
*** Fitted performance in 77.86815333366394s ***


/home/adam/dev/ilqr/ilqr/controller.py:171: UserWarning: exceeded max regularization term
  warnings.warn("exceeded max regularization term")


# Receding horizon

In [7]:
for k, ilqr in ilqrs.items():
    underlying = 'receding_' + k
    dynamics[underlying] = dynamics[k]
    x0s[underlying] = x0s[k]
    xs[underlying], us[underlying] = receding(env, ilqr, x0s[k], us_init)

iteration 0 r = 0.12829613031170972, x = [[-0.02517132 -0.00313229  0.08415416 -0.0036109   0.01121883  0.03684529
  -0.07181501 -0.0241356  -0.06113325  0.04126975]
 [-0.02471767 -0.00485111  0.08509381 -0.00552478  0.01079747  0.0538995
  -0.27157668  0.21161663 -0.32123058 -0.12506055]], u = [[-3.9749622 -0.8965623]]
iteration 1 r = 0.07130554605429325, x = [[-0.02471767 -0.00485111  0.08509381 -0.00552478  0.01079747  0.0538995
  -0.27157668  0.21161663 -0.32123058 -0.12506055]
 [-0.02425912 -0.00663831  0.08609261 -0.0076334   0.01071637  0.0377886
  -0.08622895 -0.01140961 -0.10087939  0.10830664]], u = [[1.10955226 1.51322329]]
iteration 2 r = 0.08352888130533441, x = [[-0.02425912 -0.00663831  0.08609261 -0.0076334   0.01071637  0.0377886
  -0.08622895 -0.01140961 -0.10087939  0.10830664]
 [-0.02394595 -0.00676315  0.08507645 -0.00782142  0.01294798  0.02489555
   0.06112873 -0.19155149  0.06310846  0.33765308]], u = [[0.84688497 0.899773  ]]
iteration 3 r = -0.0039365510281045

iteration 25 r = -0.3188837030403183, x = [[-0.02413623 -0.09188525  0.12754782 -0.23377589  0.44694118 -0.03079216
  -1.01316874  0.86032388 -2.09513889  2.43407405]
 [-0.02452912 -0.10129532  0.13536748 -0.25359238  0.47108332 -0.04794227
  -0.8702975   0.70511829 -1.87053066  2.39543773]], u = [[0.49214965 0.44575819]]
iteration 26 r = -0.2368756260038722, x = [[-0.02452912 -0.10129532  0.13536748 -0.25359238  0.47108332 -0.04794227
  -0.8702975   0.70511829 -1.87053066  2.39543773]
 [-0.0251612  -0.10879654  0.14108436 -0.27047149  0.49501493 -0.07864957
  -0.63188092  0.44030059 -1.5081471   2.39126958]], u = [[0.96038222 0.78684777]]
iteration 27 r = -0.21985471119288125, x = [[-0.0251612  -0.10879654  0.14108436 -0.27047149  0.49501493 -0.07864957
  -0.63188092  0.44030059 -1.5081471   2.39126958]
 [-0.02611268 -0.11457615  0.14480873 -0.28497234  0.51991347 -0.11180302
  -0.52412867  0.30488134 -1.39107937  2.58445123]], u = [[0.6492489  1.17856216]]
iteration 28 r = -0.2547699

iteration 51 r = -0.16906679174674968, x = [[-0.01122008 -0.34542973  0.36868341 -0.70690938  0.79345649  0.49029439
  -1.28724415  1.55758527 -1.73941135 -1.36507334]
 [-0.00602423 -0.35863193  0.38479518 -0.72464169  0.77883026  0.54871433
  -1.35101425  1.66256324 -1.80412084 -1.55892479]], u = [[-0.8648142  -1.94815464]]
iteration 52 r = -0.11774798052852897, x = [[-6.02423356e-03 -3.58631930e-01  3.84795176e-01 -7.24641693e-01
   7.78830258e-01  5.48714326e-01 -1.35101425e+00  1.66256324e+00
  -1.80412084e+00 -1.55892479e+00]
 [-5.93169125e-04 -3.70387177e-01  3.99619184e-01 -7.39261811e-01
   7.60691035e-01  5.36332417e-01 -1.00445508e+00  1.30695997e+00
  -1.12653534e+00 -2.06583591e+00]], u = [[ 1.02574373 -4.81834688]]
iteration 53 r = -0.15804038045674576, x = [[-5.93169125e-04 -3.70387177e-01  3.99619184e-01 -7.39261811e-01
   7.60691035e-01  5.36332417e-01 -1.00445508e+00  1.30695997e+00
  -1.12653534e+00 -2.06583591e+00]
 [ 4.71872703e-03 -3.81435219e-01  4.13690870e-01 -7

iteration 76 r = -0.04064257558645717, x = [[ 0.12246899 -0.62458638  0.72386521 -1.14718383  0.63468128  0.25750142
  -0.69469819  0.78731136 -1.51619034  0.97215764]
 [ 0.12467361 -0.63125069  0.73119454 -1.16222839  0.6459885   0.18325091
  -0.63832972  0.67878922 -1.49242192  1.28750694]], u = [[0.39934179 0.90937835]]
iteration 77 r = -0.06433160780750001, x = [[ 0.12467361 -0.63125069  0.73119454 -1.16222839  0.6459885   0.18325091
  -0.63832972  0.67878922 -1.49242192  1.28750694]
 [ 0.12646392 -0.63686075  0.73715701 -1.17500847  0.65602874  0.17538676
  -0.48553381  0.51553867 -1.06741151  0.72458998]], u = [[ 0.90230923 -0.96137758]]
iteration 78 r = -0.10977440637687602, x = [[ 0.12646392 -0.63686075  0.73715701 -1.17500847  0.65602874  0.17538676
  -0.48553381  0.51553867 -1.06741151  0.72458998]
 [ 0.12810088 -0.64160842  0.74212763 -1.18559248  0.6636827   0.15197972
  -0.46406579  0.47865895 -1.04939977  0.80592472]], u = [[0.12496206 0.25589915]]
iteration 79 r = -0.124

iteration 102 r = -0.10647900628368805, x = [[ 0.11321881 -0.68412607  0.75352886 -1.3168824   0.89690989  0.02960817
   0.3638611  -0.51040926  1.12375508 -1.4738334 ]
 [ 0.11354564 -0.67987908  0.74777429 -1.30392651  0.87998447  0.035265
   0.48611535 -0.64063368  1.46756467 -1.91037711]], u = [[  3.98151287 -11.74882771]]
iteration 103 r = -0.06798384384278129, x = [[ 0.11354564 -0.67987908  0.74777429 -1.30392651  0.87998447  0.035265
   0.48611535 -0.64063368  1.46756467 -1.91037711]
 [ 0.11391249 -0.67439042  0.74071187 -1.28752691  0.85872108  0.03743539
   0.61237471 -0.77216038  1.81263835 -2.34130422]], u = [[ 2.68660264 -4.64324632]]
iteration 104 r = -0.09438190695374557, x = [[ 0.11391249 -0.67439042  0.74071187 -1.28752691  0.85872108  0.03743539
   0.61237471 -0.77216038  1.81263835 -2.34130422]
 [ 0.1142792  -0.66761467  0.73232295 -1.26766648  0.83317931  0.03506197
   0.74371034 -0.90613168  2.15989144 -2.76595264]], u = [[ 3.29256497 -9.96875412]]
iteration 105 r = 

...

iteration 891 r = -0.2939262073656384, x = [[ 1.62434697  0.60695037 -1.92247639  1.6345797   1.02636881  0.28427871
  -0.06429216 -0.33503003  0.24391529  0.15207659]
 [ 1.62615717  0.60627973 -1.92468659  1.63488049  1.03044327  0.07806365
  -0.06965696 -0.10736865 -0.18283872  0.66126854]], u = [[-5.67914284  1.87176556]]
iteration 892 r = -0.2338590109194847, x = [[ 1.62615717  0.60627973 -1.92468659  1.63488049  1.03044327  0.07806365
  -0.06965696 -0.10736865 -0.18283872  0.66126854]
 [ 1.62665977  0.60571589 -1.92515193  1.6326189   1.03615246  0.0225046
  -0.04314073  0.01419783 -0.26959944  0.48137565]], u = [[-3.60931905 -3.24003611]]
iteration 893 r = -0.2729448032582321, x = [[ 1.62665977  0.60571589 -1.92515193  1.6326189   1.03615246  0.0225046
  -0.04314073  0.01419783 -0.26959944  0.48137565]
 [ 1.62660829  0.6054157  -1.92440582  1.62948757  1.04008391 -0.03271331
  -0.0170003   0.13475686 -0.35658867  0.30542449]], u = [[ -2.88829168 -11.44081434]]
iteration 894 r = -

iteration 917 r = 0.0022422036788432072, x = [[ 1.55554326  0.60755721 -1.80071482  1.4415138   1.09413375 -0.36219774
  -0.05491803  0.46231389 -0.91065281  0.80078954]
 [ 1.55223067  0.60695192 -1.79671691  1.43295611  1.10286731 -0.30045535
  -0.06584033  0.33772912 -0.80096019  0.94502012]], u = [[1.00046905 1.01335055]]
iteration 918 r = 0.04342788094179189, x = [[ 1.55223067  0.60695192 -1.79671691  1.43295611  1.10286731 -0.30045535
  -0.06584033  0.33772912 -0.80096019  0.94502012]
 [ 1.54954144  0.60625503 -1.79393046  1.42552116  1.11283227 -0.23742461
  -0.0733511   0.21975743 -0.68592531  1.04716515]], u = [[0.92024416 0.84696895]]
iteration 919 r = -0.019656307630784377, x = [[ 1.54954144  0.60625503 -1.79393046  1.42552116  1.11283227 -0.23742461
  -0.0733511   0.21975743 -0.68592531  1.04716515]
 [ 1.54728945  0.60547012 -1.79211076  1.41886682  1.12419518 -0.21294037
  -0.083507    0.14424991 -0.64455167  1.22391493]], u = [[0.75453531 1.07811988]]
iteration 920 r = 0.0

iteration 943 r = -0.030594379865261243, x = [[ 1.45780026  0.57294824 -1.77872379  1.22101242  1.65564171 -0.0224187
  -0.09909262 -0.33080666 -0.13595264  1.28682748]
 [ 1.45732578  0.57196688 -1.78148048  1.21909453  1.66773271 -0.07218256
  -0.09717014 -0.22139923 -0.24705566  1.13328323]], u = [[-0.84710791 -2.96514633]]
iteration 944 r = -0.06339792341940623, x = [[ 1.45732578  0.57196688 -1.78148048  1.21909453  1.66773271 -0.07218256
  -0.09717014 -0.22139923 -0.24705566  1.13328323]
 [ 1.45782673  0.57108406 -1.78495785  1.21873666  1.67703589  0.17158781
  -0.0797765  -0.47312094  0.17378428  0.72976604]], u = [[  0.96228764 -23.46761198]]
iteration 945 r = -0.11005275839590281, x = [[ 1.45782673  0.57108406 -1.78495785  1.21873666  1.67703589  0.17158781
  -0.0797765  -0.47312094  0.17378428  0.72976604]
 [ 1.45952199  0.57027396 -1.78983445  1.22054958  1.68506083  0.16730794
  -0.08222649 -0.50174175  0.18853476  0.87402608]], u = [[0.58642627 1.0220347 ]]
iteration 946 r 

iteration 969 r = -0.14260136123589065, x = [[ 1.66337441  0.53825878 -2.09773524  1.58189515  1.69918201  1.12122793
  -0.23452293 -1.55693365  2.13038892 -0.54068397]
 [ 1.67523841  0.53578072 -2.11397399  1.60443241  1.69215298  1.24961545
  -0.2612855  -1.6886882   2.37412475 -0.86338719]], u = [[25.62948924 -4.67803217]]
iteration 970 r = -0.17535340618001335, x = [[ 1.67523841  0.53578072 -2.11397399  1.60443241  1.69215298  1.24961545
  -0.2612855  -1.6886882   2.37412475 -0.86338719]
 [ 1.68654164  0.53334379 -2.12940671  1.6260089   1.68525585  1.01460873
  -0.22582095 -1.40237875  1.94690551 -0.5181349 ]], u = [[-9.59520951  1.08966749]]
iteration 971 r = -0.18006612706437403, x = [[ 1.68654164  0.53334379 -2.12940671  1.6260089   1.68525585  1.01460873
  -0.22582095 -1.40237875  1.94690551 -0.5181349 ]
 [ 1.69624156  0.53127186 -2.14254473  1.64452449  1.67950336  0.92723371
  -0.18853011 -1.22797924  1.75948837 -0.63223437]], u = [[-7.56349562 -2.98624538]]
iteration 972 r 

iteration 994 r = -0.48046018656965706, x = [[ 1.75003139  0.54144091 -2.18112814  1.72828708  1.6238793  -0.32309441
   0.25061007  0.68783907 -0.78080637  0.17092208]
 [ 1.74705736  0.54370438 -2.17493795  1.72113503  1.62623042 -0.27199783
   0.20274712  0.55142126 -0.65060745  0.29891842]], u = [[ 9.85585335 17.86916013]]
iteration 995 r = -0.5238857523397186, x = [[ 1.74705736  0.54370438 -2.17493795  1.72113503  1.62623042 -0.27199783
   0.20274712  0.55142126 -0.65060745  0.29891842]
 [ 1.74416645  0.54588368 -2.16894662  1.71419203  1.62854409 -0.30598601
   0.2326529   0.64599381 -0.7373157   0.16415694]], u = [[-0.9208112  -7.60762972]]
iteration 996 r = -0.4388563433541046, x = [[ 1.74416645  0.54588368 -2.16894662  1.71419203  1.62854409 -0.30598601
   0.2326529   0.64599381 -0.7373157   0.16415694]
 [ 1.74069496  0.54831463 -2.16192106  1.7060064   1.63059346 -0.38813158
   0.2529938   0.7582971  -0.89913786  0.2457876 ]], u = [[-0.81856976 -0.02127326]]
iteration 997 r = 

# Rollouts

In [8]:
multi_evaluate(env, dynamics, x0s, us)

performance
receding_performance


,rewards,lengths
performance,143.173724,1000
receding_performance,108.720677,1000
